In [6]:
import pandas as pd
import numpy as np

import requests

import json

from datetime import datetime, timedelta

from pprint import  pprint



In [7]:
lang = input('¿En español (es) o en ingles (en)?')
category = input('¿Balance, demanda, generacion o intercambio?')
widget = input('¿Qué widget quieres?')

restaDia = float(input('¿Cuantos días atrás?:____________'))
input_año = int(input('¿Qué año?:_____________'))

In [8]:
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es"
}

In [16]:
def generacion_datos(lang, category, widget):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')
    
    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')
    
    query = f'start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day'


    endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
    response = requests.get(url= endpoint, headers= headers)
    data = response.json()

    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()
    lista_dias = list()
    lista_meses = list()
    lista_años = list()



    for dato in data['included']:
        nombre = dato['attributes']['title']
        tipo = dato['attributes']['type']


        for i in dato['attributes']['values']:
            valor = i['value']

            porcentaje = i['percentage']

            fecha = i['datetime']
            fecha = pd.to_datetime(fecha)
            dia = fecha.strftime('%d')
            mes = fecha.strftime('%m')
            año = fecha.strftime('%Y')

            fecha =  fecha.strftime("%d/%m/%Y")

            lista_nombres.append(nombre)
            lista_tipos.append(tipo)
            lista_valores.append(valor)
            lista_porcentajes.append(porcentaje)
            lista_fechas.append(fecha)
            lista_dias.append(dia)
            lista_meses.append(mes)
            lista_años.append(año)

    df_generacion = pd.DataFrame()
    df_generacion['nombre']               = lista_nombres
    df_generacion['tipo de energía']      = lista_tipos
    df_generacion['Valores']              = lista_valores
    df_generacion["Porcentaje"]           = lista_porcentajes
    df_generacion["Fecha actualización"]  = lista_fechas
    df_generacion["dia"]                  = lista_dias               
    df_generacion["mes"]                  = lista_meses               
    df_generacion["año"]                  = lista_años


    return df_generacion
    
df_generacion = generacion_datos(lang, category, widget)

df_generacion

,nombre,tipo de energía,Valores,Porcentaje,Fecha actualización,dia,mes,año
0,Hidráulica,Renovable,24335.723,0.039181,23/09/2023,23,09,2023
1,Hidráulica,Renovable,23312.579,0.038861,24/09/2023,24,09,2023
2,Hidráulica,Renovable,38970.565,0.057330,25/09/2023,25,09,2023
3,Hidráulica,Renovable,40808.219,0.059450,26/09/2023,26,09,2023
4,Hidráulica,Renovable,44279.829,0.061009,27/09/2023,27,09,2023
...,...,...,...,...,...,...,...,...
131,Generación total,Generación total,686431.637,1.000000,26/09/2023,26,09,2023
132,Generación total,Generación total,725787.110,1.000000,27/09/2023,27,09,2023
133,Generación total,Generación total,722285.065,1.000000,28/09/2023,28,09,2023
134,Generación total,Generación total,697873.118,1.000000,29/09/2023,29,09,2023


In [17]:
df_generacion.to_csv('generacion_estructura.csv')